In [1]:
!pip install transformers
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

In [2]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import TrainingArguments, DataCollatorForLanguageModeling, Trainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
# Define 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
#model_id = "lmsys/vicuna-7b-v1.3"

In [5]:
# Load the pre-trained causal language model with quantization
model_id = "lmsys/vicuna-7b-v1.3"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [6]:
# Enable Gradient Checkpointing and Prepare for k-bit Training
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


In [7]:
#model

In [8]:
# Apply LoRA Configuration
config = LoraConfig(
    r=8, 
    lora_alpha=32, 
    target_modules=["q_proj","k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], 
    lora_dropout=0.05, 
    bias="none", 
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, config)

In [9]:
#!nvidia-smi

In [11]:
# Load and preprocess data
file_path = '/kaggle/input/wikipedia-summary-subset1k/df_subset_50000_with_prompt_filtered.csv'
# df = pd.read_csv(file_path)
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenized_prompts = tokenizer.batch_encode_plus(df['prompt'].tolist(), truncation=True, padding=False)
# prompt_lengths = [len(tokens) for tokens in tokenized_prompts['input_ids']]
df = pd.read_csv(file_path)
filtered_df = df[:1000]

In [12]:
len(filtered_df)

1000

In [ ]:
#DO ENTIRE DATASET AND LIMIT TO A 10000 FIRST OR 1K FIRST. 

In [13]:
# Tokenize the prompts and create the targets
input_data = tokenizer.batch_encode_plus(filtered_df['prompt'].tolist(), truncation=True, padding=True, return_tensors='pt')
input_ids = input_data['input_ids']
attention_mask = input_data['attention_mask']
labels = input_ids[:, 1:].clone()
labels[labels == tokenizer.pad_token_id] = -100

# Create a TensorDataset for training
train_dataset = TensorDataset(input_ids[:, :-1], attention_mask[:, :-1], labels)

# Define a custom data collator
class CustomDataCollator:
    def __call__(self, batch):
        input_ids, attention_mask, labels = zip(*batch)
        return {
            "input_ids": torch.stack(input_ids),
            "attention_mask": torch.stack(attention_mask),
            "labels": torch.stack(labels),
        }

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4, # could do 4 
    gradient_accumulation_steps=8, # could do 8, giving an effective batch size of 32. Do max of 16. Keep increasing train batch size until memory runs out
    num_train_epochs=5, # Train for 5 epochs
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    save_total_limit=3, # Limit the number of checkpoint files to keep
    save_steps=250, # Save a checkpoint every 250 steps
    output_dir="/kaggle/working/outputs",
    report_to=[] # Disable wandb integration if needed
)

# Create the Trainer
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    args=training_args,
    data_collator=CustomDataCollator(),
)

# Set use_cache to False
model.config.use_cache = False

# Start training
trainer.train()


Step,Training Loss
1,13.399800
2,12.176900
3,11.198300
4,9.555400
5,8.528200
6,7.384700
7,6.698500
8,6.465000
9,6.296300
10,6.064100


TrainOutput(global_step=155, training_loss=3.3425499685349003, metrics={'train_runtime': 32538.4603, 'train_samples_per_second': 0.154, 'train_steps_per_second': 0.005, 'total_flos': 1.0318637137526784e+17, 'train_loss': 3.3425499685349003, 'epoch': 4.96})

In [14]:
import shutil

# Save the model
save_path = "/kaggle/working/my_model"
model.save_pretrained(save_path)

# Save the tokenizer
tokenizer.save_pretrained(save_path)

# Compress the saved model directory
shutil.make_archive("/kaggle/working/my_model.zip", 'zip', save_path)

print("Model saved and compressed.")


Model saved and compressed.
